**Trần Thành Long 
19021323**

# 1. Mô tả bài toán

- Đặt vấn đề 
    -
>     Home Depot là một website bán hàng , khách hàng khi vào web site có thể nhập tên sản phẩm mong muốn vào ô search để tìm. Bài toán đặt ra trong cuộc thi đó chính là cải thiện trải nghiệm mua sắm của khách hàng bằng cách phát triên một mô hình dự đoán và tối ưu hóa đầu ra là `Search_relevance` để có thể gợi ý sản phẩm sao cho liên quan nhất với những gì khách hàng mong muốn tìm kiếm nhằm tiết kiệm thời gian tìm kiếm của khách hàng

- Mô tả dữ liệu:
    -
- Các file data :  
    - `Train.csv` : tập train gồm sản phẩm (product), tìm kiếm (search) và mức độ liên quan (relevance)
    - `Test.csv`: tập test gồm sản phẩm (product) , tìm kiếm (search) và dựa vào tập test này chúng ta phải dự đoán được mức độ liên quan (relevance)
    - `Product_descriptions.csv`: chứa đoạn text mô tả sản phẩm
    - `Attributes.csv`: chứa thông số thuộc tính của sản phẩm, một số sản phẩm không có thuộc tính
- Các trường data :
    - `Id`: id của một cặp search_term và sản phẩm.
    - `Product_uid`: id của sản phẩm
    - `Product_title` : tiêu đề của sản phẩm 
    - `Product_description` : mô tả sản phẩm 
    - `Search_term`: đoạn text được ghi lại
    - `Relevance` : mức độ liên quan search và kết quả tìm kiếm 
    - `Name`: tên thuộc tính
    - `Value`: giá trị thuộc tính



# 2. Phân tích dữ liệu

## 2.1 Khai báo các thư viện

In [ ]:
import pandas as pd 
import re 
import math
from nltk.stem.snowball import SnowballStemmer # kỹ thuật stemmer đưa về từ gốc
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

import lxml
import re
import nltk
from nltk.corpus import stopwords # các stopwords (the, or, and, ...)
from nltk.metrics import edit_distance
from string import punctuation
from collections import Counter
import numpy as np 

## 2.2 Phân tích dữ liệu

In [ ]:
#Giải nén tập tin
!unzip ../input/home-depot-product-search-relevance/attributes.csv.zip
!unzip ../input/home-depot-product-search-relevance/product_descriptions.csv.zip
!unzip ../input/home-depot-product-search-relevance/test.csv.zip
!unzip ../input/home-depot-product-search-relevance/train.csv.zip

In [ ]:
#Đọc dữ liệu
train_data = pd.read_csv("train.csv", encoding="ISO-8859-1")
test_data = pd.read_csv("test.csv", encoding="ISO-8859-1")
attribute_data = pd.read_csv('attributes.csv')
descriptions = pd.read_csv('product_descriptions.csv')

### 2.2.1 Train_dataset

In [ ]:
#Tập train 
train_data.head(10)

In [ ]:
train_data.shape

In [ ]:
train_data.info()

- Tập train gồm 74067 hàng và 5 cột trong đó chứa các trường : `product_uid` , `product_tile` , `search_term` và `relevance`
 


In [ ]:
# Thống kê số lượng mức độ liên quan trong tập train 
def count_relevance():
    high = [i for i in train_data['relevance'] if i >= 3.00]
    medium = [i for i in train_data['relevance'] if i > 1.00 and i < 3.00]
    low = [i for i in train_data['relevance'] if i == 1.00]
    return [len(high), len(medium) , len(low)]

In [ ]:
from matplotlib import pyplot as plt

In [ ]:

plt.bar(['high', 'medium', 'low'], count_relevance(), label='Relevance score', color='red')
plt.xlabel('name')
plt.ylabel('numbers')
plt.title('Bar')
plt.legend()
plt.show()

### 2.2.2 Test_dataset 

In [ ]:
test_data.shape

In [ ]:
test_data.head(10)

In [ ]:
test_data.info()

- Tập test có 166693 hàng và 4 cột 
- Chứa các trường `product_uid` , `product_tile` , `search_term` 
- Dựa vào tập test này để mô hình dự đoán `relevance`

### 2.2.3 Attribute_dataset

In [ ]:
attribute_data.shape

In [ ]:
attribute_data.head(50)

In [ ]:
attribute_data.info()

- Tập attribute chứa 2044803 hàng và 3 cột , gồm các trường `product_uid`, `name` và `value`
- Với mỗi `product_uid` có một name khác nhau thì value cũng khác nhau tuy nhiên các name khác nhau chỉ một hoặc hai ký tự.

In [ ]:
attribute_counts = attribute_data.name.value_counts()
print(attribute_counts)
print(attribute_data['name'].unique())
print(len(attribute_data['name'].unique()))

- Nhìn vài kết quả, ta thấy nhiều tên thuộc tính là Bullet, không mang ý nghĩa gì. Tuy nhiên lại có những thông tin mang ngữ nghĩa.
- Có một trường thông tin là MFG Brand Name xuất hiện nhiều nhất, ngữ nghĩa của nó là trường mang thông tin nhà sản xuất sản phẩm đó (Manufacturing Brand Name) 
- Có 5411 trường thông tin khác nhau, trên tổng số 2 triệu mẫu, tức là ta hoàn toàn có khả năng dựa vào thông tin này để lấy đặc trưng

### 2.2.4 Descriptions_dataset

In [ ]:
descriptions.shape

In [ ]:
descriptions.head(10)

In [ ]:
descriptions.info()

- Tập descriptions chứa 124428 hàng và 2 cột, gồm các trường `product_uid` và `product_description` 
- Trường `product_description` được sử dụng để search nên trường này khá quan trọng 
- Trường mô tả là text có chứa cả chữ số , có các ký tự bị viết tắt và đặc biệt khá dài

# 3.Tạo lại dữ liệu và Clean dữ liệu

## 3.1 Kiểm tra dữ liệu trùng nhau

In [ ]:
#train 
print("train :" ,train_data[train_data.duplicated(keep='first')].shape )
print("test : ", test_data[test_data.duplicated(keep='first')].shape )
print("attribute : ", attribute_data[attribute_data.duplicated(keep='first')].shape )
print("descriptions : ", descriptions[descriptions.duplicated(keep='first')].shape )


- Như vậy ngoài tập Attribute có chứa 154 hàng trùng nhau ra, các tập còn lại không có dữ liệu bị trùng nhau

In [ ]:
#Kiểm tra các hàng trùng nhau
attribute_data[attribute_data.duplicated(keep='first')]

- Như ta nhận thấy các hàng trùng nhau đều có giá trị NaN vì thế giá trị này sẽ được loại bỏ 

In [ ]:
# Loại bỏ các hàng trùng nhau và kiểm tra lại 
attribute_data = attribute_data.drop_duplicates()
descriptions[descriptions.duplicated(keep='first')].shape 

- Đã loại bỏ xong các hàng trùng nhau

## 3.2 Phân bổ relevance 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import norm  

train_data.relevance.plot(kind='hist', density=True)
mu, std = norm.fit(train_data.relevance)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
plt.title(title)
plt.show()

- Từ đồ thị ta thấy được điểm `relevance` phân bổ chủ yếu từ 2-3
- Đồ thị trên không phải là phân bố chuẩn 

## 3.3 Tạo tập data mới 

- Như đã nói trong phần phân tích Attribute_data , MFG là một thông tin hữu ích
- Ta muốn xây dựng một dataFrame khác chỉ dựa trên thông tin này, bằng việc chọn những hàng ở attribute_data có attribute_data.name == "MFG Brand Name", lấy product_uid và "value", và trường value được thay bằng thông tin mới: brand - tên nhãn hiệu

In [ ]:
# Tách tập dữ liệu brand từ tập Attribute
df_brand = attribute_data[attribute_data.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
df_brand.head(5)

**Xây dựng attribute_data mới**

- Hiện ta muốn xây dựng một trường attribute_data mới, chỉ gồm 2 trường là product_uid và attribute (text)
- Thông tin "Bullet" là vô nghĩa và sẽ xóa chúng.
- Ta tạo attribute_data_stripped mới (để tránh thay đổi dữ liệu gốc), với hàm re.sub để xóa đi các "Bullet", thay bằng xâu rỗng

In [ ]:
df_attr_stripped = attribute_data
df_attr_stripped['name'] = df_attr_stripped['name'].astype(str)
df_attr_stripped['name'] = df_attr_stripped['name'].apply(lambda s: re.sub(r"Bullet([0-9]+)", "", s))
df_attr_stripped.head(10)

- Sau bước bên trên, giờ attribute sẽ cần mang dữ liệu từ cả name và value, do đó ta cộng xâu lại.
- Ta xây dựng trường attribute bằng việc cộng xâu, với khoảng trắng ở giữa, giữa tên attribute và trường value.

In [ ]:
df_attr_stripped['attribute'] = df_attr_stripped['name'] + " " + df_attr_stripped['value']
df_attr_stripped.head(10)

- Sau bước trên, ta vẫn thấy có vấn đề, đó là một product_uid chứa nhiều attribute.
- Do đó, ta nhóm hết các dữ liệu bằng hàm groupby('product_uid'), sau đó cộng xâu chúng lại bằng hàm ' '.join(...) với agg(), phân tách bởi dấu cách.
- Kết quả là attribute_data_new, với 2 trường là product_uid và attribute, sẵn sàng cho việc gộp vào dữ liệu chính all_data như brand_data và descriptions.

In [ ]:
df_attr_final = df_attr_stripped.groupby('product_uid').agg({'attribute': lambda s : ' '.join(s.astype(str))}).reset_index()
df_attr_final.head(20)

> Ta thấy được tập Train và Test đều có những trường giống nhau và chỉ khác nhau ở việc Test không có trường relevance 
Do đó ta ghép chúng với nhau để dễ dàng xử lý 

In [ ]:
data_all = pd.concat([train_data, test_data], axis=0, ignore_index=True) 
data_all.head(5)

- Sau đó ta nối các dữ liệu khác như descriptions , attribute , brand vào `data_all`

In [ ]:
data_all = pd.merge(data_all, descriptions, how = 'left', on = 'product_uid')

In [ ]:
data_all = pd.merge(data_all, df_brand, how = 'left', on = 'product_uid')

In [ ]:
data_all = pd.merge(data_all, df_attr_final, how = 'left', on = 'product_uid')

In [ ]:
data_all.head(10)

- Như vậy việc tạo data mới đã xong giờ ta sẽ làm sạch dữ liệu để model có thể học 1 cách tối ưu

## 3.4 Clean dữ liệu

### 3.4.1 Tiền xử lý



- Chuẩn hóa xâu : 
    - Thay thế các kí tự, chuỗi không cần thiết bằng khoảng trắng
    - Chuyển chữ thành số ví dụ chuyển fivexfive->5x5
    - Chuyển chữ hoa thành chữ thường

- Ta xây dựng một hàm với input là 1 xâu đầu vào , kết quả trả về một xâu đã được chuẩn hóa 

In [ ]:
def string_edit(s:str): 
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) 
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") 
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        
        # Consolidate variations of equivalent unit terms 
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        
        # Handling numeric instances with common identifiers
        s = re.sub(r"zero\.?", r"0 ", s)
        s = re.sub(r"one\.?", r"1 ", s)
        s = re.sub(r"two\.?", r"2 ", s)
        s = re.sub(r"three\.?", r"3 ", s)
        s = re.sub(r"four\.?", r"4 ", s)
        s = re.sub(r"five\.?", r"5 ", s)
        s = re.sub(r"six\.?", r"6 ", s)
        s = re.sub(r"seven\.?", r"7 ", s)
        s = re.sub(r"eight\.?", r"8 ", s)
        s = re.sub(r"nine\.?", r"9 ", s)
        
        return s
    else:
        # Return a "null" string if the parameter supplied is not a string
        return "null"

- Dùng hàm chuẩn hóa đã định nghĩa ở trên và chuẩn hóa `all_data` chuyển hết chữ hoa thành chữ thường, chữ thành số 

In [ ]:
import time

start_time = time.time()

data_all['search_term'] = data_all['search_term'].apply(str).apply(string_edit)
data_all['product_title'] = data_all['product_title'].apply(str).apply(string_edit)
data_all['product_description'] = data_all['product_description'].apply(str).apply(string_edit)
data_all['attribute'] = data_all['attribute'].apply(str).apply(string_edit)
print(data_all.head())

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
data_all['brand'] = data_all['brand'].apply(str).apply(string_edit)

In [ ]:
data_all.head(10)

### 3.4.2 Loại bỏ các stopwords

- Stopword là những từ ví dụ như the , in , on , at , which , what , .....
- Sử dụng hàm stopwords trong thư viện nltk để xử lý
- Định nghĩa hàm str_steamer để loại bỏ stopword

In [ ]:
from bs4 import BeautifulSoup
import lxml
import re
import nltk
from nltk.corpus import stopwords # Import the stop word list
from nltk.metrics import edit_distance
from string import punctuation
from collections import Counter

def str_stemmer(doc):
    # thêm khoảng trắng để tách các từ
    tokens = doc.split()
    # loại bỏ các dấu chấm câu
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # loại bỏ các từ không nằm trong bảng chữ cái
    tokens = [word for word in tokens if word.isalpha()]
    # lọc các từ là stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # lọc ra các từ ngắn 
    tokens = [word for word in tokens if len(word) > 1]
    return ' '.join(tokens)

def str_stemmer_title(s):
    return " ".join(map(stemmer.stem, s.lower().split()))

def str_common_word(str1, str2):
    whole_set = set(str1.split())
    return sum(int(str2.find(word)>=0) for word in whole_set)

def get_shared_words(row_data):
    return np.sum([str_common_word(*row_data[:-1]), str_common_word(*row_data[1:])])

In [ ]:
data_all['search_term'] = pd.Series(data_all['search_term'].map(lambda x:str_stemmer(str(x))))
data_all['product_title'] = pd.Series(data_all['product_title'].map(lambda x:str_stemmer(str(x))))
data_all['product_description'] = pd.Series(data_all['product_description'].map(lambda x:str_stemmer(str(x))))
data_all['attribute'] = pd.Series(data_all['attribute'].map(lambda x:str_stemmer(str(x))))

In [ ]:
data_all.head(10)

### 3.4.3 Loại bỏ các thẻ HTML

In [ ]:
print('total data has html tags in',descriptions.product_description.str.count('<br$').values.sum())

- Như ta thấy có 173 tag HTML trong các đoạn text
- Trong quá trình thu thập dữ liệu từ các trang web thì còn sót lại các tag HTML mà chúng không cần thiết cho quá trình phân tích dữ liệu hay huấn luyện mô hình nên ta cần xem xét và loại bỏ
- Sử dụng BeautifulSoup để loại bỏ tag html

In [ ]:
def remove_html_tag(text):
    soup = BeautifulSoup(text, 'lxml')
    text = soup.get_text().replace('Click here to review our return policy for additional information regarding returns', '')
    return text

In [ ]:
data_all['search_term'] = pd.Series(data_all['search_term'].map(lambda x:remove_html_tag(str(x))))
data_all['product_title'] = pd.Series(data_all['product_title'].map(lambda x:remove_html_tag(str(x))))
data_all['product_description'] = pd.Series(data_all['product_description'].map(lambda x:remove_html_tag(str(x))))

In [ ]:
data_all.head(10)

# 4. Trích chọn đặc trưng  

## 4.1 Các dữ liệu dạng văn bản

- Tách các đoạn văn thành các từ

In [ ]:
data_all['search_term_tokens'] = data_all.search_term.str.lower().str.split()
data_all['product_title_tokens'] = data_all.product_title.str.lower().str.split()
data_all['product_description_tokens'] = data_all.product_description.str.lower().str.split()

- Ta thu được thêm 3 đặc trưng mới đo là `search_term_tokens`, `product_title_tokens`, `product_description_tokens`

In [ ]:
data_all.head(10)

## 4.2 Tính edit_distance

- Edit_distance là số bước cần thực hiện để chuyển từ chuỗi s1 -> s2. Ví dụ để chuyển rain -> shine cần 3 bước:"rain” -> “sain” -> “shin” -> “shine” => edit_distance = 3. Ta sẽ dùng hàm edit_distance của thư viện nltk để tính ra đặc trưng edit_distance giữa 2 cặp

In [ ]:
data_all["edistance_sprot"] = [edit_distance(word1, word2) for word1, word2 in
                                    data_all[["search_term","product_title"]].values.tolist()]
data_all["edistance_sd"] = [edit_distance(word1, word2) for word1, word2 in
                                    data_all[["search_term","product_description"]].values.tolist()]

- Sau bước trên ta thu được 2 đặc trưng mới:
    - edistance_sprot là khoảng cách edit_distance giữa search_term và product_title
    - edistance_sd là khoảng cách edit_distance giữa search_term và product_description

In [ ]:
data_all.head(10)

## 4.3 Tính Jaccard simulator

- Jaccard simulator dùng để tính độ tương đồng giữa 2 đoạn văn

In [ ]:
def get_jaccard_sim(columns): 
    str1, str2 = columns[0], columns[1]
    a = set(str1) 
    b = set(str2)
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

- Dùng hàm get_jaccard_sim để sinh ra 2 đặc trưng mới là jaccard simulator giữa (search_term, product_title) và (search_term và product_description)

In [ ]:
data_all['j_dis_sqt'] = [get_jaccard_sim(rows) for rows in data_all[["search_term_tokens","product_title_tokens"]].values]
data_all['j_dis_sqd'] = [get_jaccard_sim(rows) for rows in data_all[["search_term_tokens","product_description_tokens"]].values]

- Thu được 2 đặc trưng mới:

    - j_dis_sqt là jaccard simulator giữa search_term_tokens và product_title_tokens
    - j_dis_sqd là jaccard simulator giữa search_term_tokens và product_description_tokens

In [ ]:
data_all.head(10)

## 4.4 Tính số từ và số chữ trong description_product, search_term, product_title, attribute

In [ ]:
data_all['search_query_length'] = data_all.search_term.str.len()
data_all['number_of_words_in_descr'] = data_all.product_description.str.count("\\w+")

In [ ]:
data_all['number_of_words_in_search'] = data_all['search_term_tokens'].map(lambda x: len(x))
data_all['number_of_words_in_title'] = data_all['product_title_tokens'].map(lambda x: len(x))
data_all['length_product_description'] = data_all.product_description.str.len()
data_all['length_product_title'] = data_all.product_title.str.len()
data_all['length_attribute'] = data_all.attribute.str.len()

In [ ]:
data_all['len_of_attribute'] = data_all['attribute'].map(lambda x:len(str(x).split())).astype(np.int64)
data_all['len_of_brand'] = data_all['brand'].map(lambda x:len(str(x).split())).astype(np.int64)

In [ ]:
data_all.head(10)

## 4.5 TF-IDF và CosineSimilary

- **TF-IDF** là viết tắt của **Term Frequency-Inverse Document Frequency**. Hiểu mộ cách đơn giản nó là sự kết hợp của tần số xuất hiện của một từ trong một mẫu và nghịch đảo của tần số của từ đó trong toàn bộ tập dữ liệu. Kỹ thuật này được phát triển để đánh giá kết quả cho các truy vấn trong công cụ tìm kiếm và hiện tại nó là một phần không thể thiếu trong xử lý ngôn ngữ tự nhiên

- **TF: Term Frequency(Tần suất xuất hiện của từ) là số lần từ xuất hiện trong văn bản :**
    - tf(t,d) đại diện cho tần số của từ t xuất hiện trong mẫu d :
    - f(t, d): Số lần xuất hiện của từ t trong văn bản d
    - max({f(w, d) : w ∈ d}): Số lần xuất hiện của từ có số lần xuất hiện nhiều nhất trong văn bản d    
     
![](https://nguyenvanhieu.vn/wp-content/uploads/2019/01/tf.png)     
     
     
        
        
 - **IDF: Inverse Document Frequency(Nghịch đảo tần suất của văn bản), giúp đánh giá tầm quan trọng của một từ :**
    - idf(t, D): giá trị idf của từ t trong tập văn bản
    - |D|: Tổng số văn bản trong tập D
    - |{d ∈ D : t ∈ d}|: thể hiện số văn bản trong tập D có chứa từ t.
    
    
![](https://nguyenvanhieu.vn/wp-content/uploads/2019/01/idf.png)    



- **Cosine Similarity (hay độ tương tự của văn bản)** là quá trình sử dụng số liệu dựa trên khoảng cách hoặc độ tương tự có thể sử dụng để xác định mức độ tương đương của một văn bản với bất kỳ văn bản nào khác dựa trên các đặc trưng được trích xuất ra từ **tf-idf**. Về cơ bản, khoảng cách cosin cung cấp cho chúng ta một số liệu biểu thị góc giữa 2 vecto đặc trưng tương ứng của từng mẫu. Khi góc bằng 0 thì cosine = 1 khi đó thì độ tương đồng là lớn nhất. 
    - Ta sử dụng hàm TfidfVectorizer của thư viện sklearn để tạo vector và tính tf-idf cho 2 cặp (search_term, product_title) và (search_term và product_description). Sau đó tính ra 2 đặc trưng mới là cosineSim_title_search và cosineSim_description.

In [ ]:
def cosineSim(v1, v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range (len(v1)):
        x = v1[i]
        y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
        
    return sumxy/math.sqrt(sumxx*sumyy)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
arr1 = []
arr2 = []
arr3 = []
for i in range (len(data_all)):
    product_title_i = data_all['product_title'][i]
    description_i = data_all['product_description'][i]
    attribute_i = data_all['attribute'][i]
    search_term_i = data_all['search_term'][i]
    
    TfidfVectorizer1  = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
    TfidfVectorizer2 = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
    TfidfVectorizer3 = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
    
    tf_idf1 = TfidfVectorizer1.fit_transform([product_title_i, search_term_i])
    tf_idf2 = TfidfVectorizer2.fit_transform([description_i, search_term_i])
    tf_idf3 = TfidfVectorizer3.fit_transform([attribute_i, search_term_i])
    
    # tinh cosine similarrities
    
    cosineSim_title_search = cosineSim(tf_idf1.toarray()[0], tf_idf1.toarray()[1])
    arr1.append(cosineSim_title_search)
    cosineSim_description = cosineSim(tf_idf2.toarray()[0], tf_idf2.toarray()[1])
    arr2.append(cosineSim_description)
    cosineSim_attribute = cosineSim(tf_idf3.toarray()[0], tf_idf3.toarray()[1])
    arr3.append(cosineSim_attribute)

In [ ]:
data_all['tfidf_cosineSim_search_title'] = arr1
data_all['tfidf_cosineSim_search_description'] = arr2
data_all['tfidf_cosineSim_search_attribute'] = arr3

- Do khi tính cosine, mẫu = 0 nên một số giá trị sẽ là NaN khiến cho tập dữ liệu không thể đưa vào huấn luyện được. Vì vậy ta sẽ thay các giá trị NaN đó bằng giá trị trung bình các cosine tính được

In [ ]:
data_all['tfidf_cosineSim_search_title'] = data_all['tfidf_cosineSim_search_title'].fillna(data_all['tfidf_cosineSim_search_title'].sum()/len(data_all))
data_all['tfidf_cosineSim_search_description'] = data_all['tfidf_cosineSim_search_description'].fillna(data_all['tfidf_cosineSim_search_description'].sum()/len(data_all))

- Sau các bước trên thu được 3 đặc trưng mới:
    - `tfidf_cosineSim_search_title` là khoảng cách cosine similary giữa 2 vector thu được sau khi thực hiện tf-idf search_term và product_title
    - `tfidf_cosineSim_search_description` là khoảng cách cosine similary giữa 2 vector thu được sau khi thực hiện tf-idf search_term và product_description
    - `tfidf_cosineSim_search_attribute` là khoảng cách cosine similary giữa 2 vector thu được sau khi thực hiện tf-idf search_term và attribute


In [ ]:
data_all.head(10)

## 4.6 Countvectorize và cosinesimilary

* CountVectorizer được sử dụng để chuyển đổi một bộ sưu tập các tài liệu văn bản thành một vectơ có số lượng thuật ngữ / mã thông báo. Nó cũng cho phép xử lý trước dữ liệu văn bản trước khi tạo biểu diễn vectơ. Chức năng này làm cho nó trở thành một mô-đun biểu diễn tính năng rất linh hoạt cho văn bản.
* Ta sử dụng countvectorize để biến text thành các ma trận thưa rồi dùng cosinesimilary để so sánh chúng

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

arr1 = []
arr2 = []
arr3 = []
for i in range (len(data_all)):
    product_title_i = data_all['product_title'][i]
    description_i = data_all['product_description'][i]
    attribute_i = data_all['attribute'][i]
    search_term_i = data_all['search_term'][i]
    
    vectorizer1 = CountVectorizer()
    vectorizer2 = CountVectorizer()
    vectorizer3 = CountVectorizer()
    
    tf1 = vectorizer1.fit_transform([product_title_i, search_term_i])
    tf2 = vectorizer2.fit_transform([description_i, search_term_i])
    tf3 = vectorizer3.fit_transform([attribute_i, search_term_i])

    
    # tinh cosine similarrities
    
    cosineSim_title_search = cosineSim(tf1.toarray()[0], tf1.toarray()[1])
    arr1.append(cosineSim_title_search)
    cosineSim_description = cosineSim(tf2.toarray()[0], tf2.toarray()[1])
    arr2.append(cosineSim_description)
    cosineSim_attribute = cosineSim(tf3.toarray()[0], tf3.toarray()[1])
    arr3.append(cosineSim_attribute)

In [ ]:
data_all['cosineSim_search_title'] = arr1
data_all['cosineSim_search_description'] = arr2
data_all['cosineSim_search_attribute'] = arr3

In [ ]:
data_all.head(10)

## 4.7 Tính ratio


### 4.7.1. Tính ratio theo công thức 
* Hàm str_common_word dùng để tìm từ chung giữa 2 đoạn văn. Ta sẽ tìm từ chung của 2 cặp (search_term, product_title) và (search_term, product_description)
* Tính 2 đặc trưng mới theo công thức sau: ratio =  $\frac{số-từ-chung}{num-of-word-in-search}$

In [ ]:
data_all['product_info'] = data_all['search_term']+'\t'+data_all['product_title']+'\t'+data_all['product_description']+'\t'+data_all['attribute']
data_all['word_in_title'] = data_all['product_info'].map(lambda x:str_common_word(str(x).split('\t')[0],str(x).split('\t')[1]))
data_all['word_in_description'] = data_all['product_info'].map(lambda x:str_common_word(str(x).split('\t')[0],str(x).split('\t')[2]))
data_all['word_in_attributes'] = data_all['product_info'].map(lambda x:str_common_word(str(x).split('\t')[0],str(x).split('\t')[3]))
data_all['ratio_title'] = data_all['word_in_title']/data_all['number_of_words_in_search']
data_all['ratio_description'] = data_all['word_in_description']/data_all['number_of_words_in_search']
data_all['ratio_attributes'] = data_all['word_in_attributes']/data_all['number_of_words_in_search']
data_all['title_length'] = data_all.product_title.str.len()

In [ ]:
data_all['attr'] = str(data_all['search_term'])+"\t"+str(data_all['brand'])
data_all['brand_in_search'] = data_all['attr'].map(lambda x:str_common_word(str(x).split('\t')[0],str(x).split('\t')[1]))
data_all['ratio_brand'] = data_all['brand_in_search']/data_all['len_of_brand']

- Tương tự như tính cosine thì tính ratio cũng sinh ra một số giá trị NaN nên phải xét giá trị cho những giá trị đó là trung bình.

In [ ]:
data_all['ratio_title'] = data_all['ratio_title'].fillna(data_all['ratio_title'].sum()/len(data_all))
data_all['ratio_description'] = data_all['ratio_description'].fillna(data_all['ratio_description'].sum()/len(data_all))
data_all['ratio_attributes'] = data_all['ratio_attributes'].fillna(data_all['ratio_attributes'].sum()/len(data_all))

- Thu được các đặc trưng mới sau:
    * `word_in_title` là số từ chung giữa search_term và product_title 
    * `word_in_description` là số từ chung giữa search_term và product_description
    * `word_in_attributes` là số từ chung giữa search_term và attribute
    * `ratio_title` là tỉ lệ số từ chung giữa search và title chia cho số từ search 
    * `ratio_description` là tỉ lệ số từ chung giữa search và desceiption chia cho số từ search 
    * `ratio_attributes` tỉ lệ số từ chung giữa search và attribute chia cho số từ search 

In [ ]:
data_all.head(10)

### 4.7.2. Tính ratio bằng hàm fuzzy

- Fuzzy Seach (tìm kiếm "mờ"), hay còn hay được gọi là Approximate Search (tìm kiếm "xấp xỉ") là khái niệm để chỉ kỹ thuật để tìm kiếm một xâu "gần giống" (thay vì "giống hệt") so với một xâu cho trước. Ví dụ: ta nhập từ tìm kiếm là "tran thanh long " có thể tìm kiếm ra từ "trn thnh log" vì 2 từ gần giống nhau 

In [ ]:
from fuzzywuzzy import fuzz
def fuzzy_partial_ratio(string_1 , string_2):
    return fuzz.partial_ratio(string_1, string_2)

In [ ]:
def fuzzy_token_sort_ratio(string_1,string_2):
    return fuzz.token_sort_ratio(string_1,string_2)

In [ ]:
data_all['fuzzy_ratio_in_title'] = data_all['product_info'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[1]))
data_all['fuzzy_ratio_in_description'] = data_all['product_info'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[2]))
data_all['fuzzy_ratio_in_attribute'] = data_all['product_info'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[3]))

In [ ]:
data_all['fuzzy_token_sort_ratio_in_title'] = data_all['product_info'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[1]))
data_all['fuzzy_token_sort_ratio_in_description'] = data_all['product_info'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[2]))
data_all['fuzzy_token_sort_ratio_in_attribute'] = data_all['product_info'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[3]))

In [ ]:
data_all.head(10)

## 4.8. Tính giá trị Mean của product_description, product_title, attribute

- Sử dụng tf-idf để biến dữ liệu dạng text thành vector sau đó tính giá trị trung bình của các vector đó để chuyển sang dạng số. Giá trị trung bình tính theo công thức sau: mean = $\frac {sum}{len(vector)}$. Ở đây ta dùng hàm np.mean()

In [ ]:
tf = TfidfVectorizer()
arr1 =[]
arr2 = []
arr3 = []
arr4 = []
for i in range (len(data_all)):
    arr1.append(tf.fit_transform([data_all['product_description'][i]]).mean())
    arr2.append(tf.fit_transform([data_all['product_title'][i]]).mean())
    arr3.append(tf.fit_transform([data_all['attribute'][i]]).mean())
data_all['mean_product_des'] = arr1
data_all['mean_product_title'] = arr2
data_all['mean_attribute'] = arr3

- Thu được 3 đặc trưng mới` mean_product_des`, `mean_product_title`, `mean_attribute`:

In [ ]:
data_all.head()

# 5. Huấn luyện mô hình

## 5.1 Tạo dữ liệu

### 5.1.1 Loại bỏ các dữ liệu dạng chữ, chỉ lấy dữ liệu dạng số để huấn luyện 

In [ ]:
data_all = data_all.drop(['product_title', 'search_term', 'product_description', 'search_term_tokens', 'product_title_tokens', 'product_description_tokens', 'product_info', 'attr', 'attribute', 'brand'], axis = 1)

In [ ]:
data_all.head(10)

### 5.1.2 Chia data_all thành 2 tập train và test 

In [ ]:
train = data_all[:74067]
test = data_all[74067:]

### 5.1.3 Chia tập dữ liệu thành X_train, y_train, X_test, id_test 

In [ ]:
y_train = train['relevance'].values
X_train = train.drop(['id', 'relevance'], axis = 1).values 
id_test = test['id']
X_test = test.drop(['id', 'relevance'], axis = 1).values

## 5.2 Huấn luyện mô hình

- Sử dụng mô hình XGBRegression (eXtreme Gradient Boosting)
- Ý tưởng của Boosting :
    - Xây dựng một lượng lớn các model (thường là cùng loại). Mỗi model sau sẽ học cách sửa lỗi của model trước để cải thiện chất lượng của mô hình (cụ thể ta sẽ giảm trọng số khi dữ liệu đúng và tăng trọng số khi dữ liệu sai)
- XGBRegression : 
    - Là thuật toán mở rộng của gradient boost
    - Được sử dụng rộng dãi trong các cuộc thi kaggle
- Ưu điểm : 
    - Dễ sử dụng, cài đặt
    - Cho độ chính xác cao
    - Các thông số có thể điều chỉnh được
    - Dữ liệu đầu vào không bị bó buộc, có thể chứa các giá trị như NaN, text,....
- Giải thích các tham số sử dụng trong mô hình :
    * max_depth: độ sâu của cây quyết định 
    * n_estimators: số lượng cây trong mô hình 
    * learning_rate: hệ số học, hệ số này dùng để nhân với kết quả mô hình trước
- Ta sử dụng feature_importances của XGBoost và matplotlib để biểu thị độ quan trọng của các feature: 


In [ ]:
import xgboost as xgb
from matplotlib import pyplot
from xgboost import plot_importance
bst = xgb.XGBRegressor(max_depth = 6,
                    n_estimators = 130).fit(X_train , y_train)
y_pred = bst.predict(X_test)
pyplot.bar(range(len(bst.feature_importances_)), bst.feature_importances_)
pyplot.show()
len(bst.feature_importances_)

In [ ]:
from xgboost import XGBRegressor
import sklearn
xgb = XGBRegressor()
param_grid = {'max_depth':[5, 6], 
              'n_estimators': [130, 150, 170], 
              'learning_rate' : [0.1]}
model_xgb = sklearn.model_selection.GridSearchCV(estimator = xgb, param_grid = param_grid, n_jobs = -1)
model_xgb.fit(X_train, y_train)

y_pred = model_xgb.predict(X_test)

- Một số giá trị dự đoán vượt quá 3 sẽ được xét lại là 3

In [ ]:
for i in range(len(y_pred)):
    if y_pred[i] > 3:
        y_pred[i] = 3

- Xuất các giá trị dự đoán tập test thành file submission.csv

In [ ]:
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)